In [45]:
import torch
import pandas as pd
import os
import sqlite3
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import OneHotEncoder

In [2]:
MATCH_ID = "978805"
EVENT_DATA_DIR = "myairbridge-6YJaD9uGRDr/machine learning data/events"
MATCH_ID_LIST = [ i.split("_")[0] for i in os.listdir(EVENT_DATA_DIR) ]
TRACKING_DATA_DIR = "myairbridge-6YJaD9uGRDr/machine learning data/tracking"

In [3]:
def load_events(m_id:str):
    events_raw = pd.read_csv(os.path.join(EVENT_DATA_DIR, f"{m_id}_events.csv"), index_col=0,)
    events = events_raw[["frame_start", "frame_end", "event_subtype"]]

    return events

In [4]:
# for m_id in MATCH_ID_LIST[:10]:
events = load_events(MATCH_ID)
    # print(events.event_type.unique())

In [5]:
tracking_raw = pd.read_csv(os.path.join(TRACKING_DATA_DIR, f"{MATCH_ID}_tracking.csv"))
tracking = tracking_raw.drop(columns=["match_id", "half", "timestamp"])
tracking_ball = tracking.loc[tracking.object_id == -1].drop(columns=["object_id", "extrapolated"])

In [6]:
conn = sqlite3.connect(':memory:')
#write the tables
events.to_sql('events', conn, index=False)
tracking_ball.to_sql('tracking_ball', conn, index=False)

qry = '''
    select 
        *
        -- x,y,z, frame_id, event_subtype
    from
        events right join tracking_ball on
        tracking_ball.frame_id between frame_start and frame_end
    '''
df = pd.read_sql_query(qry, conn)

In [9]:
X = df[['x', 'y', 'z']]
y = df['event_subtype']

In [ ]:
y.value_counts().plot(kind='bar')

In [56]:
X_tensor = torch.tensor(X.values, dtype=torch.float32).unsqueeze(0)

In [57]:
classes = y.unique()
n_classes = len(classes)

In [44]:
torch.tensor(y.values)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [87]:
ohe = OneHotEncoder()
y_ohe = torch.tensor(ohe.fit_transform(y.values.reshape(-1, 1)).toarray()).unsqueeze(0)

In [26]:
lstm = nn.LSTM(3, 3, bidirectional=True)
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))

In [62]:
bilstm = BiLSTM(3, 3, n_classes)

In [33]:
X_example = torch.randn(1, 10, 3)

In [83]:
out = bilstm(X_tensor[:,:10000])

In [84]:
out.shape

torch.Size([1, 10000, 11])

In [89]:
criterion = nn.CrossEntropyLoss()
loss = criterion(out, y_ohe[:,:10000])

In [27]:
class BiLSTM(nn.Module):

    def __init__(self, input_dim:int, hidden_dim:int, batch_size:int, output_dim:int=11, num_layers:int=2):
        super(BiLSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        #Define the initial linear hidden layer
        self.init_linear = nn.Linear(self.input_dim, self.input_dim)

        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers, batch_first=True, bidirectional=True)

        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim * 2, output_dim)

    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input):
        #Forward pass through initial hidden layer
        linear_input = self.init_linear(input)

        # Forward pass through LSTM layer
        # shape of lstm_out: [batch_size, input_size ,hidden_dim]
        # shape of self.hidden: (a, b), where a and b both
        # have shape (batch_size, num_layers, hidden_dim).
        lstm_out, self.hidden = self.lstm(linear_input)

        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out)
        return y_pred

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
